In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sys

In [ ]:
sys.path.append("../input/timm-pytorch-image-models/pytorch-image-models-master")

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
from PIL import Image
import os
import torch.nn as nn
import torch.nn.functional as F
import math
import albumentations as A
import time
from albumentations.pytorch import ToTensorV2
from torchvision import models
import copy
import timm

In [ ]:
device=torch.device('cuda'if torch.cuda.is_available() else 'cpu')
device2=torch.device('cpu')

test_path='/kaggle/input/petfinder-pawpularity-score/test/'
submission_path='/kaggle/input/petfinder-pawpularity-score/sample_submission.csv'



testcsv=pd.read_csv("/kaggle/input/petfinder-pawpularity-score/test.csv")
submission=pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')

testcsv['img_path']=testcsv['Id']+'.jpg'

In [ ]:
class dataini(Dataset):
    def __init__(self,rootdir,dataframe,transform=None):
        self.root_dir=rootdir
        self.transform=transform
        self.dataframe=dataframe
    def __len__(self):
        return len(self.dataframe)
    def __getitem__(self,index):
        if torch.is_tensor(index):
            index = index.tolist()
        
        img_name = self.root_dir+ self.dataframe.iloc[index,-1]
        image = Image.open(img_name)
        label = torch.tensor(self.dataframe.iloc[index, -2],dtype=torch.float32)
        
    
        
        
        if self.transform:
            imagenp=np.array(image)
            augmented = self.transform(image=imagenp)
            image = augmented['image']
            
    
        return image, label

In [ ]:
transforms1=A.Compose([A.Resize(256, 256), 
    A.RandomCrop(224, 224),
    A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ToTensorV2(),
    ])

In [ ]:
test_set=dataini(test_path,testcsv,transform=transforms1)
test_loader=DataLoader(dataset=test_set,batch_size=1)

In [ ]:
model = torch.load('../input/model1/model1.pth',map_location=device)

model.eval()

In [ ]:
output=[]
for image, label in test_loader:
    
    image=image.to(device)
    
    outputtest=model(image).to(device2)
    outputtest=torch.squeeze(outputtest)
    outputtest=outputtest.detach().numpy()
    output.append(outputtest)
    
    

In [ ]:
output=np.array(output)
output=np.squeeze(output)

In [ ]:
my_submission = pd.DataFrame({'Id': testcsv.Id, 'Pawpularity':output })
my_submission.to_csv('submission.csv', index=False)

In [ ]:
my_submission